In [1]:
from transformers import MarianMTModel, MarianTokenizer, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import DatasetDict , load_dataset, Dataset, concatenate_datasets
import os 
#import evaluate
import pandas as pd
os.environ["WANDB_DISABLED"] = "true"

2024-06-17 10:27:05.614823: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 10:27:05.614923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 10:27:05.745270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
adjectives = pd.read_csv("/kaggle/input/asjectives/adjectives.csv")
adverbs = pd.read_csv("/kaggle/input/adverbs/adverbs.csv")
prepositions = pd.read_csv("/kaggle/input/prepostionnouns/prepositions.csv")
nouns = pd.read_csv("/kaggle/input/prepostionnouns/prepositions.csv")


adjectives = adjectives.rename(columns={"n1": "darija", "eng":"en"})
prepositions = prepositions.rename(columns={"n1": "darija", "eng":"en"})
nouns = nouns.rename(columns={"n1": "darija", "eng":"en"})
adverbs = adverbs.rename(columns={"n1": "darija", "eng":"en"})

dataset_adjectives = Dataset.from_pandas(adjectives)
dataset_prepositions = Dataset.from_pandas(prepositions)
dataset_nouns = Dataset.from_pandas(nouns)
dataset_adverbs = Dataset.from_pandas(adverbs)

print(len(adjectives) + len(adverbs) + len(prepositions) + len(nouns))

1182


In [3]:
dataset_ar_eng1 = load_dataset("atlasia/darija-translation")
dataset_ar_eng2 = load_dataset("BounharAbdelaziz/English-to-Moroccan-Darija")
dataset_ar_eng2 = dataset_ar_eng2.rename_column("english", "en")


dataset_ar_eng = concatenate_datasets([dataset_ar_eng1["train"], dataset_ar_eng2["train"] ,dataset_adjectives, dataset_adverbs, dataset_nouns, dataset_prepositions])
dataset_ar_eng = dataset_ar_eng.train_test_split(test_size=0.2)

# Filter out rows with NaN values
dataset_ar_eng = DatasetDict({
    'train': dataset_ar_eng['train'].filter(lambda x: x['darija'] is not None and x['en'] is not None),
    'test': dataset_ar_eng['test'].filter(lambda x: x['darija'] is not None and x['en'] is not None)
})



dataset_ar_eng = DatasetDict({
    'train': dataset_ar_eng['train'].remove_columns([col for col in dataset_ar_eng['train'].column_names if col not in ['darija', 'en']]),
    'test': dataset_ar_eng['test'].remove_columns([col for col in dataset_ar_eng['test'].column_names if col not in ['darija', 'en']])
})

dataset_ar_eng

Generating train split:   0%|          | 0/45103 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/16089 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49899 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12475 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['darija', 'en'],
        num_rows: 49578
    })
    test: Dataset({
        features: ['darija', 'en'],
        num_rows: 12407
    })
})

In [4]:
model_name = "marefa-nlp/marefa-mt-en-ar"
tokinezer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def process_data(examples):
    prefix = "translate English to arabic: "
    inputs = [prefix + example for example in examples["en"]]
    targets = [example for example in examples['darija']]
    
    model_inputs = tokinezer(inputs, max_length=128, truncation=True, padding="max_length")
    
    with tokinezer.as_target_tokenizer():
        labels = tokinezer(targets, max_length=128, truncation=True, padding="max_length")
        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
tokinezed_dataset_ar_en = dataset_ar_eng.map(process_data, batched=True, remove_columns=dataset_ar_eng["train"].column_names)

Map:   0%|          | 0/49578 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/12407 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorForSeq2Seq(tokinezer, model=model)

In [39]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # BLEU score
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    # ROUGE score
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    
    # Combine metrics
    result = {
        "bleu": bleu_result["bleu"],
        "rouge1": rouge_result["rouge1"].mid.fmeasure,
        "rouge2": rouge_result["rouge2"].mid.fmeasure,
        "rougeL": rouge_result["rougeL"].mid.fmeasure,
        "rougeLsum": rouge_result["rougeLsum"].mid.fmeasure,
    }
    
    return result

In [8]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokinezed_dataset_ar_en["train"],
    eval_dataset=tokinezed_dataset_ar_en["test"],
    tokenizer=tokinezer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.398500,0.356238
2,0.330500,0.307416
3,0.297300,0.282857
4,0.276900,0.269981
5,0.263200,0.262747
6,0.256800,0.260168


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=9300, training_loss=0.33730128462596604, metrics={'train_runtime': 4689.5066, 'train_samples_per_second': 63.433, 'train_steps_per_second': 1.983, 'total_flos': 1.0083678818402304e+16, 'train_loss': 0.33730128462596604, 'epoch': 6.0})

In [54]:
# English Sample Text
input = "do you realy love me"

translated_tokens = model.generate(**tokinezer.prepare_seq2seq_batch([input], return_tensors="pt").to("cuda"))
translated_text = [tokinezer.decode(t, skip_special_tokens=True) for t in translated_tokens]

# translated Arabic Text
print(translated_text)

['wach katbghini tari9i']
